In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv', index_col=0)

# Transforming train df

In [ ]:
def convert_df(df):
    new_df = {}
    df_values = df.values

    for i in range(0, len(df), 65):
        temp_df = df_values[i:i+65]
        t = df_values[i, 1]
        new_df[t] = {str((x, y, d)): c for (x, y, d, c) in temp_df[:, 2:6]}
    new_df = pd.DataFrame(new_df).transpose()
    new_df.index = pd.to_datetime(new_df.index)
    
    return new_df

In [ ]:
def compute_features(df):
    df['weekday'] = df.index.weekday
    df['date'] = df.index.date
    df['time'] = df.index.time
    df['hour'] = df.index.hour
    return df

In [ ]:
train = convert_df(train)
TARGET_COLS = deepcopy(train.columns)
train = compute_features(train)
FEATURES = [c for c in train.columns if c not in TARGET_COLS]
train.head()

In [ ]:
daily_congestion = train[train.hour<12].groupby('date').mean()[TARGET_COLS].mean(axis=1)
all_congestion = train.groupby('time').median()
all_congestion

# Submission

In [ ]:
sample_submission.congestion = all_congestion[TARGET_COLS].iloc[36:].values.flatten()
sample_submission.to_csv('average baseline.csv')
sample_submission